<a href="https://colab.research.google.com/github/nerofinal012/-https-github.com-MicrosoftLearning-eShopOnWeb/blob/main/Cisco_Dataset_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Install Required Dependencies
!pip install transformers[torch]
!pip install accelerate -U

# Step 2: Restart Runtime (do this manually in Colab)
# Runtime > Restart runtime

# Step 3: Verify Installation
import transformers
import accelerate

print(transformers.__version__)
print(accelerate.__version__)

# Step 4: Import Required Libraries
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, pipeline
import pandas as pd

4.41.2
0.31.0


In [2]:
# Load the dataset from Hugging Face
cisco_dataset = load_dataset("ndavidson/cisco_dcnm_finetuning_chatml", split="train")
print(cisco_dataset.column_names)  # Inspect the columns

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


['Question', 'Text', 'Answer', 'prompt_and_answer']


In [3]:
# Create a DataFrame from the dataset
df_cisco = pd.DataFrame(cisco_dataset)

# Preprocessing: Add input and response columns
df_cisco['input'] = df_cisco['Question']
df_cisco['response'] = df_cisco['Answer']

# Display the first few rows to verify
print(df_cisco[['input', 'response']].head())


                                               input  \
0  what is the latest version of Nexus Dashboard ...   
1  newest Nexus Dashboard Fabric Controller version?   
2  what's current Nexus Dashboard Fabric Controll...   
3   latest Nexus Dashboard Fabric Controller update?   
4     Nexus Dashboard Fabric Controller version now?   

                                            response  
0  The latest version of Nexus Dashboard Fabric C...  
1  Based on the information provided in the sourc...  
2  Based on the information provided in the sourc...  
3  Based on the information provided in the sourc...  
4  The current version of the Nexus Dashboard Fab...  


In [4]:
# Choose a pre-trained model
model_name = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Assign a padding token
tokenizer.pad_token = tokenizer.eos_token

# Define the tokenization function
def tokenize_function(examples):
    return tokenizer(examples['input'], truncation=True, padding='max_length', max_length=512)

# Convert the DataFrame back to a Dataset object
cisco_dataset = Dataset.from_pandas(df_cisco)

# Apply the tokenization function
tokenized_datasets = cisco_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/2506 [00:00<?, ? examples/s]

In [5]:
# Define the tokenization function
def tokenize_function(examples):
    return tokenizer(examples['input'], truncation=True, padding='max_length', max_length=512)

# Convert the DataFrame back to a Dataset object
cisco_dataset = Dataset.from_pandas(df_cisco)

# Apply the tokenization function
tokenized_datasets = cisco_dataset.map(tokenize_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

Map:   0%|          | 0/2506 [00:00<?, ? examples/s]

In [6]:
# Create the Trainer Instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets
)


In [ ]:
# Train the Model
trainer.train()

# Evaluate the Model
results = trainer.evaluate()
print(results)

In [ ]:
# Deploy the Chatbot
chatbot = pipeline("conversational", model=model, tokenizer=tokenizer)

In [ ]:
# Test the chatbot
response = chatbot("Hello, how can I help you today?")
print(response)